In [1]:
import pandas as pd
import numpy as np
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chaha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chaha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chaha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
df_true = pd.read_csv("C:/Users/chaha/Desktop/NewsProject/NewsDataset/True.csv")
df_fake = pd.read_csv("C:/Users/chaha/Desktop/NewsProject/NewsDataset/Fake.csv")

In [5]:
df_fake = df_fake[["title", "text"]].dropna()
df_true = df_true[["title", "text"]].dropna()

# Add labels
df_fake["label"] = 0
df_true["label"] = 1

# Combine title + text
df_fake["combined"] = df_fake["title"] + " " + df_fake["text"]
df_true["combined"] = df_true["title"] + " " + df_true["text"]

# Merge
df = pd.concat([df_fake, df_true], ignore_index=True)
df = df[["combined", "label"]].dropna().sample(frac=1, random_state=42).reset_index(drop=True)


In [6]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w.isalpha() and w not in stop_words]
    return ' '.join(tokens)

df['cleaned'] = df['combined'].apply(preprocess)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned'], df['label'], test_size=0.2, random_state=42)

In [8]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [9]:
model = LogisticRegression()
model.fit(X_train_vec, y_train)

LogisticRegression()

In [10]:
y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9856347438752784
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4710
           1       0.98      0.99      0.98      4270

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [13]:
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

In [12]:
joblib.dump(model, "logistic_model.pkl")

['logistic_model.pkl']